In [2]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import xlsxwriter

In [3]:
# URLs to scrape
urls = ['https://www.youtube.com/@chess/videos']

In [4]:
# Initialize variables
times = 0
row = 0
t, v, d = [], [], []

service = Service(executable_path=r"C:\Program Files (x86)\Google\chromedriver-win64\chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Loop through each URL
for url in urls:
    driver.get(f'{url}/videos?view=0&sort=p&flow=grid')
    times = 0
    while times < 5:
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        times += 1
    content = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content, 'html.parser')

# Extract video titles
title_elements = soup.findAll('yt-formatted-string', id='video-title')
for title in title_elements:
    t.append(title.text.strip()) 

# Extract views
view_elements = soup.findAll('span', class_='inline-metadata-item style-scope ytd-video-meta-block')
for view in view_elements:
    if 'просмотров' in view.text:
        v.append(view.text.strip().replace("\u00a0тыс.", "K").replace("\u00a0млн", "M"))

# Extract duration
duration_elements = soup.findAll('span', class_='style-scope ytd-thumbnail-overlay-time-status-renderer')
for duration in duration_elements:
    aria_label = duration.get('aria-label')
    if aria_label and any(char.isdigit() for char in aria_label):
        d.append(aria_label.strip())

driver.quit()


In [5]:
# Creating an Excel workbook and worksheet
workbook = xlsxwriter.Workbook('file.xlsx')
worksheet = workbook.add_worksheet()

# Writing headers
worksheet.write(0, 0, "Title")
worksheet.write(0, 1, "Views")
worksheet.write(0, 2, "Duration")

# Writing data to the worksheet
row = 1
for title, view, duration in zip(t, v, d):
    worksheet.write(row, 0, title)
    worksheet.write(row, 1, view)
    worksheet.write(row, 2, duration)
    row += 1

workbook.close()

In [11]:
import pandas as pd

data = pd.read_excel('file.xlsx')
data.head()

,Title,Views,Duration
0,HIKARU'S REAL THOUGHTS ABOUT LEVY REVEALED 😞 |...,391K просмотров,40:18
1,Chess Prodigy Bodhana Sivananadan Meets Judit ...,2 дня назад,40:18
2,3 Minutes Of Hikaru Premoving Checkmate,22K просмотров,20:07
3,GRANDMASTERS GUESS CHESS POSITIONS WITHOUT SEE...,7 дней назад,20:07
4,THE WORST CHESS ADVICE?! GM REACTS!,31K просмотров,3:43


In [12]:
# Remove extra characters from the Views column 
data['Views'] = data['Views'].str.replace(" просмотров", "").str.replace(" ", "")
new_views = []

for i in data['Views']:
    if 'K' in i:
        i = i.replace('K', '')
        new_views.append(float(i) * 1000)
    elif 'M' in i:
        i = i.replace('M', '')
        new_views.append(float(i) * 1000000)
    else:
        new_views.append(i)

data['View'] = new_views

ValueError: could not convert string to float: '9,8'